## Model Zoo | Classify your own data

In [ ]:
import torchvision
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
def my_image_show ( image ) :
    
#     plt.figure(figsize = (10,10))
    plt.imshow(image, aspect='auto')
    plt.axis('off')
    plt.title('My image.')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def my_tensor_image_show ( image , label=None ):
    image = image.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)
    plt.imshow(image)
    plt.axis('off')
    if label is None :
        plt.title('Image in tensor format.')
    else :
        plt.title(f'Image in tensor format | Class: {label:2d}')
    plt.show()    

In [ ]:
def my_raw_tensor_image_show ( image , label=None ):
    image = image.numpy().transpose((1, 2, 0))
    plt.imshow(image)
    plt.axis('off')
    if label is None :
        plt.title('Image in tensor format.')
    else :
        plt.title(f'Image in tensor format | Class: {label:2d}')
    plt.show()    

### Load image using OpenCV

In [ ]:
import cv2

dataset_path = '/homeLocal/praticas-cv-cnn/datasets/'
name = dataset_path + '/car.jpeg'


image = cv2.imread( name )
image = cv2.cvtColor( image , cv2.COLOR_BGR2RGB )
my_image_show(image)

print(type(image))
print(image.shape)

pil_image = Image.fromarray(image)
print(type(pil_image))

### Load image using PIL

In [ ]:
dataset_path = '/homeLocal/praticas-cv-cnn/datasets/'
name = dataset_path + '/car.jpeg'

pil_image = Image.open( name )
my_image_show(pil_image)

print(type(pil_image))
print(pil_image.size, pil_image.getbands())

### Load image using Decode_image from torchvision io

In [ ]:
from torchvision.io import decode_image

dataset_path = '/homeLocal/praticas-cv-cnn/datasets/'
name = dataset_path + '/car.jpeg'

img = decode_image(name)

my_raw_tensor_image_show(img)

# In this case, img is already a tensor. I just renamed it to pil_image to maintaing consistency along the code.
pil_image = img

### Process the data and classify it

In [ ]:
from torchvision.models import alexnet, AlexNet_Weights
import torch

my_transform = AlexNet_Weights.IMAGENET1K_V1.transforms()
model_input = my_transform(pil_image) 

my_tensor_image_show(model_input)
print(model_input.shape)

# model_input = my_transform(pil_image) 
model_input = model_input.unsqueeze_(0)
print(model_input.shape)

if torch.cuda.is_available():
    my_device = torch.device("cuda:0")    
else:
    my_device = torch.device("cpu")

my_device = torch.device("cpu")

print(f"Running on {my_device.type}.")

model = torchvision.models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
model = model.to(my_device)
model_input = model_input.to(my_device)

model.eval()
with torch.no_grad():
    output = model(model_input)
    
print(output.shape)

In [ ]:
## Show the image class

prediction = output.squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = AlexNet_Weights.IMAGENET1K_V1.meta["categories"][class_id]
print(f"Class: {category_name} with probability {100 * score:5.2f}%")

In [ ]:
## Show the first top-k image class probabilities

prediction = output.squeeze(0).softmax(0)
  
# Show top categories per image
top5_prob, top5_catid = torch.topk(prediction, 10)

for i in range(top5_prob.size(0)):
    category_name = AlexNet_Weights.IMAGENET1K_V1.meta["categories"][top5_catid[i]]
    print( f'Class: {category_name} with probability {100*top5_prob[i].item():5.2f}%' )

## Model Zoo | Describle your own data using a deep descriptor

Visit https://docs.pytorch.org/vision/stable/models.html to a list of available models.

Click in the desired model for more details. Then click in the GitHub link of the model. 

Create a Class that extends the model class, and overwrite the atribute on the classifier 

In [ ]:
from torchvision.models import AlexNet

class AlexNetDescriptor(AlexNet):
    def __init__(self):
        super(self.__class__, self).__init__()
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            # torch.nn.Dropout(),
            # torch.nn.Linear(4096, 4096),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
model_descriptor = AlexNetDescriptor()
model_descriptor.eval()

In [ ]:
model.eval()

In [ ]:
# This is the manner I explained during the class. However, in the new version of PyTorch,
# the same result can be achieved using the bellow cell.

descriptor_dict = model_descriptor.state_dict()
pretrained_dict = model.state_dict()

# 1. filter out keys not presented in the new model
keys_dict = { k: v for k, v in pretrained_dict.items() if k in descriptor_dict }
# 2. overwrite entries in the existing state dict
descriptor_dict.update(keys_dict) 
# 3. load the new state dict
model_descriptor.load_state_dict(descriptor_dict)
print(descriptor_dict.keys())

#### You can load the weights using only the below code, instead of running the more complex code presented in the above cells. Using this approach, it is not necessary to load the dictionary of both models, compare the keys, filter out the keys, and load the new dictionary in the descriptor model.



In [ ]:
from torchvision.models import AlexNet

class AlexNetDescriptor(AlexNet):
    def __init__(self):
        super(self.__class__, self).__init__()
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),
            torch.nn.Linear(4096, 4096),
            # torch.nn.ReLU(inplace=True),
            # torch.nn.Linear(4096, 1000),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model_descriptor = AlexNetDescriptor()

# Directly load weights
model_descriptor.load_state_dict(torchvision.models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1).state_dict(), strict=False)    


In [ ]:
model_input = my_transform(pil_image) 
model_input = model_input.unsqueeze_(0)
print(model_input.shape)

if torch.cuda.is_available():
    my_device = torch.device("cuda:0")
else:
    my_device = torch.device("cpu")

print(f"Running on {my_device.type}.")

model_descriptor = model_descriptor.to(my_device)
model_input = model_input.to(my_device)

model_descriptor.eval()
with torch.no_grad():
    output = model_descriptor(model_input)
    
print(output.shape)